In [1]:
%matplotlib inline
import os, sys, time
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [4]:
# load the raw data
df_train = pd.read_csv('./data/train.csv')
print('Shape of the dataset: {}'.format(df_train.shape))
df_train.head()

Shape of the dataset: (595212, 59)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [3]:
# is there class imbalance? The answer is very much...
df_train['target'].value_counts()

0    573518
1     21694
Name: target, dtype: int64

In [18]:
# what features are we looking at?
# there are 57 features in total (=59-2)
all_features = df_train.columns.tolist()
all_features.remove('target')
all_features.remove('id')
bin_features = []
cat_features = []
other_features = []
for col in df_train.columns:
    if col.split('_')[-1] == 'bin':
        bin_features.append(col)
    elif col.split('_')[-1] == 'cat':
        cat_features.append(col)
    else:
        other_features.append(col)

In [10]:
len(bin_features)

17

In [11]:
len(cat_features)

14

In [39]:
# how many missing values are we looking at?
df_train[['target']+all_features].groupby('target')[all_features]\
.apply(lambda x: 100*np.sum(x == -1)/len(x)).T

target,0,1
ps_ind_01,0.000000,0.000000
ps_ind_02_cat,0.030688,0.184383
ps_ind_03,0.000000,0.000000
ps_ind_04_cat,0.008718,0.152116
ps_ind_05_cat,0.928480,2.231032
ps_ind_06_bin,0.000000,0.000000
ps_ind_07_bin,0.000000,0.000000
ps_ind_08_bin,0.000000,0.000000
ps_ind_09_bin,0.000000,0.000000
ps_ind_10_bin,0.000000,0.000000


In [28]:
df_train[['target']+all_features].groupby('target')['ps_calc_15_[]bin'].sum()

target
0    70232
1     2638
Name: ps_calc_15_bin, dtype: int64

In [12]:
# let me build a vanila random forest model, as a bench mark
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

t_start = time.time()